Hyperparameter optimization is done in 2 groups of 3 parameters each. 3 values are evaluated for each parameter. This requires 54 iterations for each group. The first group optimizes parameters for the convolutional layer and the second group optimizes other model parameters. To reduce search time, the procedure uses only 2 cross-validation folds and 3 epochs. This should be sufficient to determine which hyperparameter combination is optimal. The achieved accuracy after the optimization is above 0.99.

In [1]:
'''Trains a simple convnet on the MNIST dataset for ONLY digits 2 and 7.
Hyperparameters are optimized in 2 groups of 3 parameters.
The optimization leads to accuracy above 0.99
'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.wrappers.scikit_learn import KerasClassifier
from keras import optimizers
from sklearn.model_selection import GridSearchCV
import numpy as np

# iteration parameters (with low values to reduce search time)
folds = 2
epochs = 3

def create_model(input_shape, 
                 optimizer='Adam',
                 learn_rate=0.01, 
                 init_mode='uniform',
                 activation='relu',
                 dropout_rate=0.25,
                 filters=4,
                 kernel_size=3,
                 print_progress=True):
    model = Sequential()
    model.add(Conv2D(filters=filters, 
                     kernel_size=(kernel_size, kernel_size),
                     activation=activation,
                     input_shape=input_shape,
                     kernel_initializer=init_mode))
    model.add(Conv2D(filters=filters*2, 
                     kernel_size=(kernel_size, kernel_size), 
                     activation=activation,
                     kernel_initializer=init_mode))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(dropout_rate))
    model.add(Flatten())
    model.add(Dense(16, activation=activation))
    model.add(Dropout(dropout_rate*2))
    model.add(Dense(2, activation='softmax'))
    selected_optimizer = getattr(optimizers, optimizer)
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=selected_optimizer(lr=learn_rate),
                  metrics=['accuracy'])
    global i
    if (print_progress):
        print('\n\nPass #%d/%d. Fold #%d/%d. Combination [ ' % 
                              (i+1, total_iter, (i%folds+1), folds), end="")
        for param in params:
            print(param+'=%s ' % eval(param), end="")    
        print(']\n', end="")
    i += 1
    return model

def grid_search(param_grid, sk_params, x_train, y_train):
    global total_iter # total iterations needed
    global i # iteration index
    global params # hyperparameters to optimize
    total_iter = np.prod([len(v) for v in param_grid.values()]) * folds
    i = 0
    params = param_grid.keys()
    model = KerasClassifier(build_fn=create_model, **sk_params, epochs=epochs)
    grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=folds, refit=False)
    print ('\nOptimizing hyperparameters: ' + str([p for p in params]), end="")
    grid_result = grid.fit(x_train, y_train)
    print('\nBest score on training set: %f using %s' % 
      (grid_result.best_score_, grid_result.best_params_))
    sk_params.update(grid_result.best_params_)
    return sk_params

def main():
    np.random.seed(1)
    batch_size = 128
    num_classes = 2
    # input image dimensions
    img_rows, img_cols = 28, 28
    # the data, shuffled and split between train and test sets
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    # only look at 2s and 7s
    train_picks = np.logical_or(y_train==2,y_train==7)
    test_picks = np.logical_or(y_test==2,y_test==7)
    x_train = x_train[train_picks]
    x_test = x_test[test_picks]
    y_train = np.array(y_train[train_picks]==7,dtype=int)
    y_test = np.array(y_test[test_picks]==7,dtype=int)
    if K.image_data_format() == 'channels_first':
        x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
        x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
        input_shape = (1, img_rows, img_cols)
    else:
        x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
        x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
        input_shape = (img_rows, img_cols, 1)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')
    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)
    # hyperparameter optimization in 2 groups of 3 parameters follows
    sk_params = dict(input_shape=input_shape) # holds model parameters
    # optimize 2DConv layer hyperparameters
    param_grid = dict(init_mode=['uniform', 'lecun_uniform', 'normal'],
                      filters=[4, 6, 8],
                      kernel_size=[1, 3, 5])
    sk_params = grid_search(param_grid, sk_params, x_train, y_train)
    # optimize model hyperparameters
    param_grid = dict(optimizer = ['Adadelta', 'Adam', 'Nadam'],
                      learn_rate = [0.001, 0.01, 0.1],
                      dropout_rate = [0.0, 0.2, 0.4])
    sk_params = grid_search(param_grid, sk_params, x_train, y_train)
    # evaluate on test set
    print ('\nEvaluating model on test set with optimized hyperparameters:')
    print ([str(k)+'='+str(v) for k,v in sk_params.items() if k not in 'input_shape'])
    sk_params.update(print_progress=False)
    model = create_model(**sk_params)
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=12,
              verbose=1,
              validation_data=(x_test, y_test))
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    
if __name__ == "__main__":
    main()

Using TensorFlow backend.


x_train shape: (12223, 28, 28, 1)
12223 train samples
2060 test samples

Optimizing hyperparameters: ['init_mode', 'filters', 'kernel_size']

Pass #1/54. Fold #1/2. Combination [ init_mode=uniform filters=4 kernel_size=1 ]
Epoch 1/3
6111/6111 [==============================] - 2s - loss: 0.2199 - acc: 0.8987     
Epoch 2/3
6111/6111 [==============================] - 1s - loss: 0.1172 - acc: 0.9501     
Epoch 3/3
5824/6111 [===========================>..] - ETA: 0s

Pass #2/54. Fold #2/2. Combination [ init_mode=uniform filters=4 kernel_size=1 ]
Epoch 1/3
6112/6112 [==============================] - 2s - loss: 0.2996 - acc: 0.8200     
Epoch 2/3
6112/6112 [==============================] - 1s - loss: 0.1522 - acc: 0.9238     
Epoch 3/3
5952/6112 [============================>.] - ETA: 0s

Pass #3/54. Fold #1/2. Combination [ init_mode=uniform filters=4 kernel_size=3 ]
Epoch 1/3
6111/6111 [==============================] - 3s - loss: 0.1188 - acc: 0.9601     
Epoch 2/3
6111/6111 [======

6112/6112 [==============================] - 4s - loss: 0.6935 - acc: 0.5036     
Epoch 2/3
6112/6112 [==============================] - 3s - loss: 0.6932 - acc: 0.5101     
Epoch 3/3
5952/6112 [============================>.] - ETA: 0s

Pass #21/54. Fold #1/2. Combination [ init_mode=uniform filters=6 kernel_size=3 ]
Epoch 1/3
6111/6111 [==============================] - 5s - loss: 0.1199 - acc: 0.9571     
Epoch 2/3
6111/6111 [==============================] - 4s - loss: 0.0641 - acc: 0.9789     
Epoch 3/3
6048/6111 [============================>.] - ETA: 0s

Pass #22/54. Fold #2/2. Combination [ init_mode=uniform filters=6 kernel_size=3 ]
Epoch 1/3
6112/6112 [==============================] - 6s - loss: 0.1657 - acc: 0.9476     
Epoch 2/3
6112/6112 [==============================] - 4s - loss: 0.0935 - acc: 0.9735     
Epoch 3/3
6112/6112 [==============================] - 2s     


Pass #23/54. Fold #1/2. Combination [ init_mode=uniform filters=6 kernel_size=5 ]
Epoch 1/3
6111/6111

6111/6111 [==============================] - 5s - loss: 0.0812 - acc: 0.9743     
Epoch 3/3
6016/6111 [============================>.] - ETA: 0s

Pass #40/54. Fold #2/2. Combination [ init_mode=uniform filters=8 kernel_size=3 ]
Epoch 1/3
6112/6112 [==============================] - 6s - loss: 0.1461 - acc: 0.9378     
Epoch 2/3
6112/6112 [==============================] - 5s - loss: 0.0877 - acc: 0.9653     
Epoch 3/3
5984/6112 [============================>.] - ETA: 0s

Pass #41/54. Fold #1/2. Combination [ init_mode=uniform filters=8 kernel_size=5 ]
Epoch 1/3
6111/6111 [==============================] - 9s - loss: 0.1231 - acc: 0.9547     
Epoch 2/3
6111/6111 [==============================] - 7s - loss: 0.0562 - acc: 0.9818     
Epoch 3/3
6048/6111 [============================>.] - ETA: 0s

Pass #42/54. Fold #2/2. Combination [ init_mode=uniform filters=8 kernel_size=5 ]
Epoch 1/3
6112/6112 [==============================] - 9s - loss: 0.1406 - acc: 0.9537     
Epoch 2/3
6112/6112 

6112/6112 [==============================] - 8s - loss: 0.0721 - acc: 0.9768     
Epoch 2/3
6112/6112 [==============================] - 7s - loss: 0.0305 - acc: 0.9895     
Epoch 3/3
6112/6112 [==============================] - 4s     


Pass #5/54. Fold #1/2. Combination [ optimizer=Nadam learn_rate=0.001 dropout_rate=0.0 ]
Epoch 1/3
6111/6111 [==============================] - 19s - loss: 0.0850 - acc: 0.9704    
Epoch 2/3
6111/6111 [==============================] - 12s - loss: 0.0256 - acc: 0.9915    
Epoch 3/3
6016/6111 [============================>.] - ETA: 0s

Pass #6/54. Fold #2/2. Combination [ optimizer=Nadam learn_rate=0.001 dropout_rate=0.0 ]
Epoch 1/3
6112/6112 [==============================] - 9s - loss: 0.0759 - acc: 0.9719     
Epoch 2/3
6112/6112 [==============================] - 7s - loss: 0.0254 - acc: 0.9921     
Epoch 3/3
6080/6112 [============================>.] - ETA: 0s

Pass #7/54. Fold #1/2. Combination [ optimizer=Adadelta learn_rate=0.01 dropout_rate=0.

6048/6112 [============================>.] - ETA: 0s

Pass #23/54. Fold #1/2. Combination [ optimizer=Nadam learn_rate=0.001 dropout_rate=0.2 ]
Epoch 1/3
6111/6111 [==============================] - 9s - loss: 0.1063 - acc: 0.9627     
Epoch 2/3
6111/6111 [==============================] - 7s - loss: 0.0389 - acc: 0.9866     
Epoch 3/3
6080/6111 [============================>.] - ETA: 0s

Pass #24/54. Fold #2/2. Combination [ optimizer=Nadam learn_rate=0.001 dropout_rate=0.2 ]
Epoch 1/3
6112/6112 [==============================] - 9s - loss: 0.1056 - acc: 0.9620     
Epoch 2/3
6112/6112 [==============================] - 8s - loss: 0.0450 - acc: 0.9861     
Epoch 3/3
6112/6112 [==============================] - 2s     


Pass #25/54. Fold #1/2. Combination [ optimizer=Adadelta learn_rate=0.01 dropout_rate=0.2 ]
Epoch 1/3
6111/6111 [==============================] - 9s - loss: 0.8006 - acc: 0.5282     
Epoch 2/3
6111/6111 [==============================] - 7s - loss: 0.6574 - acc: 0.599

Epoch 1/3
6111/6111 [==============================] - 10s - loss: 0.3395 - acc: 0.8189    
Epoch 2/3
6111/6111 [==============================] - 8s - loss: 0.2107 - acc: 0.9064     
Epoch 3/3
6048/6111 [============================>.] - ETA: 0s

Pass #42/54. Fold #2/2. Combination [ optimizer=Nadam learn_rate=0.001 dropout_rate=0.4 ]
Epoch 1/3
6112/6112 [==============================] - 11s - loss: 0.3005 - acc: 0.8316    
Epoch 2/3
6112/6112 [==============================] - 8s - loss: 0.1853 - acc: 0.9040     
Epoch 3/3
6112/6112 [==============================] - 3s     


Pass #43/54. Fold #1/2. Combination [ optimizer=Adadelta learn_rate=0.01 dropout_rate=0.4 ]
Epoch 1/3
6111/6111 [==============================] - 11s - loss: 0.9046 - acc: 0.5004    
Epoch 2/3
6111/6111 [==============================] - 8s - loss: 0.8516 - acc: 0.5140     
Epoch 3/3
6111/6111 [==============================] - 3s     


Pass #44/54. Fold #2/2. Combination [ optimizer=Adadelta learn_rate=0.01